In [1]:
from bs4 import BeautifulSoup
import lxml #is a parser for HTMl
import webscraping
import requests
import os
import re
import csv
#from selenium import webdriver
from langdetect import detect
import pandas as pd

We shoul generalize these functions... in webscraping.py!

## **[RQ1.1] Get the list of books**

We start to scrape the three pages to extract the URLs for each of them

In [32]:
UrlsFiles = open("urlpages.txt", "w")

url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+str(1)
page = requests.get(url)
soup = BeautifulSoup(page.content, features='lxml')
for a in soup.find_all('a', class_="bookTitle"): #.contents[0].split('\n')[1].strip()
    UrlsFiles.write(a.get('href')+'\n')

#UrlsFiles.truncate(UrlsFiles.tell()-1) see how to remove the '\n'
UrlsFiles.close()

## **RQ[1.2] Crawl books**

In [10]:
headpart = "https://www.goodreads.com"

In [11]:
direct = "htmlpages" #Directory  
parent_dir = "./" #Parent Directory path 
pathAncestor = os.path.join(parent_dir, direct) #Path
os.mkdir(pathAncestor) #create the folder in the path

In [12]:
for i in range(1,101):
    os.makedirs(os.path.join(pathAncestor, 'page ' + str(i)))

In [13]:
UrlsFiles = open("urlpages.txt", "r")

for i, x in enumerate(UrlsFiles,1):
    subdirectory = pathAncestor + "/page " + str(i)
    article_name = "/article_"+str(i)+".html"
    
    complete_path = subdirectory + article_name
    with open(complete_path, "wb") as ip_file:
        link = headpart + x
        page = requests.get(link)
        
        soup = BeautifulSoup(page.text, features='lxml')
        
        ip_file.write(soup.encode('utf-8'))
        ip_file.close()

UrlsFiles.close()

## **RQ[1.3] Parse downloaded pages**

In [44]:
def scrap_book(href, tsv_writer, Url):
    r = requests.get(href) #call to have the html page with the useful information for my analysis
    soup = BeautifulSoup(r.text, features="lxml") #parse the text
    
    #extract rating and review count
    ratings = soup.find_all('a', href="#other_reviews") #search the ratings in its
    rating_count = -1
    rating = -1
    for raiting in ratings:
        if raiting.find_all('meta', itemprop="ratingCount"):
            ratingCount = raiting.text.replace('\n', '').strip().split(' ')[0].replace(',', '')
        elif raiting.find_all('meta', itemprop="reviewCount"):
            reviewCount = raiting.text.replace('\n', '').strip().split(' ')[0].replace(',', '')
    
    #extract the book title
    bookTitle = soup.find_all('h1')[0].contents[0].replace('\n', '').strip()

    #extract the book authors
    bookAuthors = soup.find_all('span', itemprop='name')[0].contents[0]
    
    #extract the book authors, we shoul FIX it.
    try:
        Plot = soup.find_all('div', id="description")[0].contents[3].text
    except:
        Plot = soup.find_all('div', id="description")
        if not Plot:
            Plot = "NaN"
        else:
            Plot = soup.find_all('div', id="description")[0].contents[1].text
    if detect(Plot) != "en":
        Plot = "NaN"
    
    #extract the date
    date = soup.find_all('div', id="details")[0].contents[3].text.replace('\n', '').strip().split()
    Published = date[1]+" "+date[2]+" "+date[3]
    
    #Rating Value
    ratingValue = soup.find('span', itemprop="ratingValue").text.strip()
    
    #Number of pages
    NumberofPages = soup.find('span', itemprop="numberOfPages").text.split()[0]
    
    #Title series
    bookSeries = soup.find_all('a', href= re.compile(r'/series/*'))
    if not bookSeries:
        bookSeries = "NaN"
    else:
        bookSeries = soup.find_all('a', href= re.compile(r'/series/*'))[0].contents[0].strip()
    
    #Places
    Setting = []
    for places in soup.find_all('a', href= re.compile(r'/places/*')):
        Setting.append(places.text)
    Setting = ", ".join(Setting) if len(Setting)>=1 else "NaN"
        
    #list of characters
    Characters = []
    for character in soup.find_all('a', href= re.compile(r'/characters/*') ):
        Characters.append(character.text)
    Characters = ", ".join(Characters) if len(Characters)>=1 else "NaN"
    
    tsv_writer.writerow([bookTitle, bookSeries, bookAuthors, ratingValue, ratingCount, reviewCount, Plot, NumberofPages, Published, Characters, Setting, Url])

In [45]:
UrlsFiles = open("urlpages.txt", "r")

with open('./output.tsv', 'w', encoding="utf-8", newline='') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(['bookTitle', 'bookSeries', 'bookAuthors', 'ratingValue', 
                    'ratingCount', 'reviewCount', 'Plot', 'NumberofPages', 'Published',
                    'Characters', 'Setting', 'Url'])
    for link in UrlsFiles:
        scrap_book("https://www.goodreads.com" + link, tsv_writer, link)

UrlsFiles.close()

## RQ[2] Search Engine

In [52]:
df1 = pd.read_csv('output.tsv',sep="\t")
df2 = pd.read_csv('output2.tsv',sep="\t")
df3 = pd.read_csv('output3.tsv',sep="\t")

complete_dataset = pd.concat([df1, df2, df3])

In [53]:
complete_dataset

,bookTitle,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot,NumberofPages,Published,Characters,Setting,Url
0,The Hunger Games,(The Hunger Games #1),Suzanne Collins,4.33,6409993,172577,"Could you survive on your own in the wild, wit...",374,September 14th 2008,"Katniss Everdeen, Peeta Mellark, Cato (Hunger ...","District 12, Panem, Capitol, Panem, Panem",/book/show/2767052-the-hunger-games\n
1,Harry Potter and the Order of the Phoenix,(Harry Potter #5),J.K. Rowling,4.50,2525510,42743,There is a door at the end of a silent corrido...,870,September 2004 by,"Sirius Black, Draco Malfoy, Ron Weasley, Petun...","Hogwarts School of Witchcraft and Wizardry, Lo...",/book/show/2.Harry_Potter_and_the_Order_of_the...
2,To Kill a Mockingbird,(To Kill a Mockingbird),Harper Lee,4.28,4528131,91816,The unforgettable novel of a childhood in a sl...,324,May 23rd 2006,"Scout Finch, Atticus Finch, Jem Finch, Arthur ...","Maycomb, Alabama",/book/show/2657.To_Kill_a_Mockingbird\n
3,Pride and Prejudice,NaN,Jane Austen,4.26,3018223,67820,Alternate cover edition of ISBN 9780679783268S...,279,October 10th 2000,"Mr. Bennet, Mrs. Bennet, Jane Bennet, Elizabet...","United Kingdom, Derbyshire, England, England, ...",/book/show/1885.Pride_and_Prejudice\n
4,Twilight,(The Twilight Saga #1),Stephenie Meyer,3.60,4990866,104922,About three things I was absolutely positive.F...,501,September 6th 2006,"Edward Cullen, Jacob Black, Laurent, Renee, Be...","Forks, Washington, Phoenix, Arizona, Washingto...",/book/show/41865.Twilight\n
...,...,...,...,...,...,...,...,...,...,...,...,...
95,The Things They Carried,NaN,Tim O'Brien,4.13,255296,14507,"In 1979, Tim O'Brien's Going After Cacciato—a ...",246,December 29th 1998,NaN,NaN,/book/show/133518.The_Things_They_Carried\n
96,Foundation,(Foundation (Publication Order) #1),Isaac Asimov,4.17,413269,9857,For twelve thousand years the Galactic Empire ...,244,June 1st 2004,"Hari Seldon, Hober Mallow, Salvor Hardin",NaN,/book/show/29579.Foundation\n
97,The Complete Works,NaN,William Shakespeare,4.50,50226,846,Tempest\tTwo Gentlemen of Verona\tMerry Wives ...,1248,September 8th 1990,NaN,NaN,/book/show/569564.The_Complete_Works\n
98,The Thirteenth Tale,NaN,Diane Setterfield,3.96,273732,23026,All children mythologize their birth...So begi...,406,September 12th 2006,"Margaret Lea, Emmeline March, Adeline March, V...",Angelfield,/book/show/40440.The_Thirteenth_Tale\n


In [54]:
#remove NaN plot
complete_dataset = complete_dataset[complete_dataset["Plot"].notna()]

## **Cleaning data**

In [98]:
#DEF Cleaning
import nltk

from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

stop = set(stopwords.words('english')) # words in english to avoid few data for the cleaning data

# We use lemmatizer and stemmer to obtain the root of the word
lemmatizer = WordNetLemmatizer() # non contesto
stemmer = PorterStemmer() # contesto

vocabulary = {}
df = complete_dataset.copy()
for i, row in enumerate(df["Plot"], 1):
    tokenizer = RegexpTokenizer("[\w']+")  # import the tokenizer punctuation
    row = tokenizer.tokenize(row.lower()) # remove the punctuation
    
    row = list(set(row) - stop) # remove the stop words
    
    row = [lemmatizer.lemmatize(word) for word in row]
    row = [stemmer.stem(word) for word in row]
    
    for word in row:
        if word in vocabulary.keys():
            if "document_"+str(i) not in vocabulary[word]:
                vocabulary[word].append("document_"+str(i))
        else:
            vocabulary[word] = ["document_"+str(i)]        

In [104]:
#create file .csv
import csv

with open('./inverted_list.tsv', 'w', encoding="utf-8", newline='') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(["Word", "Term_id", "Document_List"])
    
    list_of_words = vocabulary.keys()
    for id, word in enumerate(list_of_words, 1):
        term_id_i = "term_id_"+str(id)
        tsv_writer.writerow([word, term_id_i, vocabulary[word]]) 

## RQ[2.1.2] Execute the query

In [136]:
query = input("Insert your query string: ")

Insert your query string: Survival YounGer


viva la patacca

In [137]:
def cleanQuery(row):
    row = row.split(" ") # split the string query
    for element in row:
        tokenizer = RegexpTokenizer("[\w']+")  # import the tokenizer punctuation
        element = tokenizer.tokenize(element.lower()) # remove the punctuation
    row = list(set(row) - stop) # remove the stop words
    
    row = [lemmatizer.lemmatize(word) for word in row]
    row = [stemmer.stem(word) for word in row]
    
    return row

In [138]:
cleanQString = cleanQuery(query)
cleanQString

['surviv', 'younger']

open the dataset vocabulary

In [139]:
inv_lst = pd.read_csv('inverted_list.tsv',sep="\t")

In [141]:
inv_lst.head()

,Word,Term_id,Document_List
0,north,term_id_1,"['document_1', 'document_46', 'document_90', '..."
1,lie,term_id_2,"['document_1', 'document_42', 'document_44', '..."
2,contend,term_id_3,"['document_1', 'document_196']"
3,katniss,term_id_4,"['document_1', 'document_223']"
4,live,term_id_5,"['document_1', 'document_21', 'document_22', '..."


In [143]:
inv_lst.set_index('Word', inplace=True)

In [145]:
inv_lst.head()

,Term_id,Document_List
Word,,
north,term_id_1,"['document_1', 'document_46', 'document_90', '..."
lie,term_id_2,"['document_1', 'document_42', 'document_44', '..."
contend,term_id_3,"['document_1', 'document_196']"
katniss,term_id_4,"['document_1', 'document_223']"
live,term_id_5,"['document_1', 'document_21', 'document_22', '..."


In [147]:
document_lists = inv_lst.loc[cleanQString, "Document_List"]

In [149]:
document_lists.values

array(["['document_1', 'document_32', 'document_45', 'document_53', 'document_54', 'document_56', 'document_68', 'document_70', 'document_77', 'document_86', 'document_151', 'document_168', 'document_177', 'document_181', 'document_188', 'document_209', 'document_214', 'document_223', 'document_230', 'document_238', 'document_263']",
       "['document_1', 'document_182']"], dtype=object)